# Day 9
## Puzzle 1

In [1]:
import numpy as np

In [2]:
input_file = 'input_1.txt'
# input_file = 'test_input_1.txt'
# input_file = 'test_input_2.txt'
# input_file = 'test_input_3.txt'
# input_file = 'test_input_4.txt'

In [3]:
with open(file=input_file, mode="r") as file:
    disk_map = file.read()

Read input line.

In [4]:
disk_map[:20]

'89298668365347703410'

In [5]:
disk_map[-20:]

'48116329972791687396'

Map the values to blocks of files and free space.

In [6]:
mapped = []
num_vals = 0  # Number of values to move.

for i, character in enumerate(disk_map):
    if i % 2 == 0:  # Even index represents blocks of files.
        mapped.extend([int(i/2)]*int(character))
        num_vals += int(character)

    else:  # Odd index represents blocks of free space.
        mapped.extend(['.']*int(character))

In [7]:
mapped[:20]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 1,
 1,
 '.']

In [8]:
mapped[-20:]

['.',
 '.',
 9998,
 9998,
 9998,
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 9999,
 9999,
 9999,
 9999,
 9999,
 9999]

Fragment the file system.

In [9]:
shifted = []
backwards_index = -1

for forwards_index, value in enumerate(mapped):
    if forwards_index == num_vals:  # If we already have moved all file fragments needed to fill the empty memory spaces.
        break

    if value == '.':  # If the space is empty.
        while mapped[backwards_index] == '.':  # Loop backwards until we find a file fragment.
            backwards_index -= 1

        shifted.append(mapped[backwards_index])  # Insert the file fragment in the empty space.
        backwards_index -= 1

    else:
        shifted.append(value)  # File fragment that doesn't need to be moved.

In [10]:
shifted[:20]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9999,
 9999,
 9999,
 9999,
 9999,
 9999,
 9998,
 9998,
 9998,
 1,
 1,
 9997]

In [11]:
shifted[-20:]

[5232,
 5232,
 5232,
 5236,
 5235,
 5235,
 5235,
 5235,
 5234,
 5234,
 5234,
 5233,
 5233,
 5233,
 5234,
 5234,
 5234,
 5234,
 5234,
 5234]

Result

In [12]:
result = int(sum(np.array(shifted)*np.arange(len(shifted))))

In [13]:
result

6241633730082

## Puzzle 2

Turn the list of mapped files and free space into a list of lists.

In [14]:
files_and_memory = []
counter = 0

for character in disk_map:
    if mapped[counter: counter + int(character)]:
        files_and_memory.append(mapped[counter: counter + int(character)])
    
    counter += int(character)

Move entire files (from the right) to the first available space (from the left).

In [15]:
files_and_memory_new = files_and_memory.copy()
current_file_number = files_and_memory[-1][0]

while current_file_number >= 0:  # Loop through all file-ID:s.
    current_file = None
    file_found = False
    m = len(files_and_memory_new)
    i = m - 1

    while not file_found:
        if files_and_memory_new[i] and files_and_memory_new[i][0] == current_file_number:
            current_file = files_and_memory_new[i]
            file_found = True

        else:
            i -= 1

    current_space = None
    enough_space_found = False
    j = 0

    while not enough_space_found and j < i:
        if len(files_and_memory_new[j]) >= len(current_file) and files_and_memory_new[j][0] == '.':
            current_space = files_and_memory_new[j]
            enough_space_found = True
        
        else:
            j += 1

    if enough_space_found:
        # Remove the file and merge empty slots if needed.
        if i == m - 1:  # The last slot/block.
            slot_before_file = files_and_memory_new[i - 1]

            if slot_before_file and slot_before_file[0] == '.':  # If the slot before is empty.
                files_and_memory_new[i - 1] = ['.']*(len(slot_before_file) + len(current_file))
                files_and_memory_new.pop(i)

            else:
                files_and_memory_new[i] = ['.']*len(current_file)

        else:
            slot_before_file = files_and_memory_new[i - 1]
            slot_after_file = files_and_memory_new[i + 1]

            if (not slot_before_file or slot_before_file[0] == '.') and (not slot_after_file or slot_after_file[0] == '.'):  # If both the slot before and after are empty.
                files_and_memory_new[i - 1] = ['.']*(len(slot_before_file) + len(current_file) + len(slot_after_file))
                files_and_memory_new.pop(i)
                files_and_memory_new.pop(i)

            elif (not slot_before_file or slot_before_file[0] == '.') and not (not slot_after_file or slot_after_file[0] == '.'):  # If the slot before is empty, but not the slot after.
                files_and_memory_new[i - 1] = ['.']*(len(slot_before_file) + len(current_file))
                files_and_memory_new.pop(i)

            elif not (not slot_before_file or slot_before_file[0] == '.') and (not slot_after_file or slot_after_file[0] == '.'):  # If the slot after is empty, but not the slot before.
                files_and_memory_new[i] = ['.']*(len(current_file) + len(slot_after_file))
                files_and_memory_new.pop(i + 1)

            else:
                files_and_memory_new[i] = ['.']*len(current_file)

        if j == i - 1:  # If the slot before the file coincides with the free space, update the size of the free space.
            current_space = files_and_memory_new[j]

        #  Insert the file and update the free space size if needed.
        if len(current_space) > len(current_file):
            files_and_memory_new[j] = ['.']*(len(current_space) - len(current_file))

        else:
            files_and_memory_new.pop(j)

        files_and_memory_new.insert(j, current_file)

    current_file_number -= 1

In [16]:
files_and_memory_new[:20]

[[0, 0, 0, 0, 0, 0, 0, 0],
 [9999, 9999, 9999, 9999, 9999, 9999],
 [9998, 9998, 9998],
 [1, 1],
 [9997, 9997, 9997, 9997, 9997, 9997, 9997, 9997],
 [9996],
 [2, 2, 2, 2, 2, 2, 2, 2],
 [9992, 9992, 9992],
 [9991],
 [9989],
 [9971],
 [3, 3, 3, 3, 3, 3],
 [9995, 9995, 9995, 9995, 9995, 9995, 9995],
 [9968],
 [4, 4, 4],
 [9987, 9987, 9987, 9987, 9987],
 [9964],
 [5, 5, 5, 5, 5],
 [9979, 9979, 9979],
 [6, 6, 6, 6]]

In [17]:
files_and_memory_new[-20:]

[[5362, 5362, 5362, 5362, 5362, 5362, 5362, 5362, 5362],
 ['.', '.', '.', '.', '.', '.'],
 [5319, 5319, 5319, 5319, 5319, 5319, 5319, 5319],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 [5322, 5322, 5322, 5322, 5322, 5322, 5322],
 ['.', '.', '.', '.', '.'],
 [5356, 5356, 5356, 5356, 5356, 5356, 5356, 5356, 5356],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 [5326, 5326, 5326, 5326, 5326, 5326, 5326],
 ['.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 [5349, 5349, 5349, 5349, 5349, 5349, 5349, 5349],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 [5347, 5347, 5347, 5347, 5347, 5347, 5347, 5347],
 ['.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 [5348, 5348, 5348, 5

Turn the list of lists into a list of shuffled files and spaces.

In [18]:
shifted_new = [value for file_or_space in files_and_memory_new for value in file_or_space]

Results.

In [19]:
result = 0

for i, value in enumerate(shifted_new):
    if value != '.':
        result += value*i

In [20]:
result

6265268809555